
## *Data Science Unit 4 Sprint 3 Assignment 1*

# Recurrent Neural Networks and Long Short Term Memory (LSTM)

![Monkey at a typewriter](https://upload.wikimedia.org/wikipedia/commons/thumb/3/3c/Chimpanzee_seated_at_typewriter.jpg/603px-Chimpanzee_seated_at_typewriter.jpg)

It is said that [**infinite monkeys typing for an infinite amount of time**](https://en.wikipedia.org/wiki/Infinite_monkey_theorem) will eventually type, among other things, the complete works of William Shakespeare. Let's see if we can get there a bit faster, with the power of Recurrent Neural Networks and LSTM.

We will focus specifically on Shakespeare's Sonnets in order to improve our model's ability to learn from the data.

In [ ]:
import random
import sys
import os

import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import LambdaCallback

from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Bidirectional
from tensorflow.keras.layers import LSTM

%matplotlib inline

# a custom data prep class that we'll be using 
from data_cleaning_toolkit_class import data_cleaning_toolkit

### Use request to pull data from a URL

[**Read through the request documentation**](https://requests.readthedocs.io/en/master/user/quickstart/#make-a-request) in order to learn how to download the Shakespeare Sonnets from the Gutenberg website. 

**Protip:** Do not over think it.

In [ ]:
# download all of Shakespears Sonnets from the Project Gutenberg website

# here's the link for the sonnets
url_shakespeare_sonnets = "https://www.gutenberg.org/cache/epub/1041/pg1041.txt"

# use request and the url to download all of the sonnets - save the result to `r`



# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# move the downloaded text out of the request object - save the result to `raw_text_data`
# hint: take at look at the attributes of `r`
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# check the data type of `raw_text_data`
type(raw_text_data)

### Data Cleaning

In [ ]:
# as usual, we are tasked with cleaning up messy data
# Question: Do you see any characters that we could use to split up the text?
raw_text_data[:3000]

In [ ]:
# split the text into lines and save the result to `split_data`
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# we need to drop all the boilder plate text (i.e. titles and descriptions) as well as white spaces
# so that we are left with only the sonnets themselves 
split_data[:20] 

**Use list index slicing in order to remove the titles and descriptions so we are only left with the sonnets.**


In [ ]:
# sonnets exists between these indicies 
# titles and descriptions exist outside of these indicies

# use index slicing to isolate the sonnet lines - save the result to `sonnets`

# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# notice how all non-sonnet lines have far less characters than the actual sonnet lines?
# well, let's use that observation to filter out all the non-sonnet lines
sonnets[200:240]

In [ ]:
# any string with less than n_chars characters will be filtered out - save results to `filtered_sonnets`

# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# ok - much better!
# but we still need to remove all the punctuation and case normalize the text
filtered_sonnets

### Use custom data cleaning tool 

Use one of the methods in `data_cleaning_toolkit` to clean your data.

There is an example of this in the guided project.

In [ ]:
# instantiate the data_cleaning_toolkit class - save result to `dctk`

# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# use data_cleaning_toolkit to remove punctuation and to case normalize - save results to `clean_sonnets`

# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# much better!
clean_sonnets

### Use your data tool to create character sequences 

We'll need the `create_char_sequenes` method for this task. However this method requires a parameter call `maxlen` which is responsible for setting the maximum sequence length. 

So what would be a good sequence length, exactly? 

In order to answer that question, let's do some statistics! 

In [ ]:
def calc_stats(corpus):
    """
    Calculates statisics on the length of every line in the sonnets
    """
    
    # write a list comprehension that calculates each sonnets line length - save the results to `doc_lens` 

    # use numpy to calcualte and return the mean, median, std, max, min of the doc lens - all in one line of code

    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# sonnet line length statistics 
mean ,med, std, max_, min_ = calc_stats(clean_sonnets)
mean, med, std, max_, min_ 

In [ ]:
# using the results of the sonnet line length statistics, use your judgement and select a value for maxlen
# use .create_char_sequences() to create sequences

# YOUR CODE HERE
raise NotImplementedError()

Take a look at the `data_cleaning_toolkit_class.py` file. 

In the first 4 lines of code in the `create_char_sequences` method, class attributes `n_features` and `unique_chars` are created. Let's call them in the cells below.

In [ ]:
# number of input features for our LSTM model
dctk.n_features

In [ ]:
# unique charactes that appear in our sonnets 
dctk.unique_chars

## Time for Questions 

----
**Question 1:** 

Why are the `number of unique characters` (i.e. **dctk.unique_chars**) and the `number of model input features` (i.e. **dctk.n_features**) the same?

**Hint:** The model that we will shortly be building here is very similar to the text generation model that we built in the guided project.

**Answer 1:**

Our model is a classification model that, when given a sequence of characters, it will predict what character should come next. We need to ask ourselves, what are all the possible options? Well, every unique character that shows up in the sonnets is a possible option. 

Therefore our model is a multi-class classification model that outputs a probability score (recall that we are using the Softmax activation function in the output layer) for each of the unique character that it can predict as the character that should show up next in the sonnet line. 

`Number of unique characters` (i.e. **dctk.unique_chars**) and the `number of model input features` (i.e. **dctk.n_features**) are the same because all unique characters are a possible outcome for our model to predict. 


**Question 2:**

Take a look at the print out of `dctk.unique_chars` one more time. Notice that there is a white space. 

Why is it desirable to have a white space as a possible character to predict?

**Answer 2:**

The model that we will be building shortly, will be given a a sequence of characters that make up a portion of a sonnet. It's task is to predict what characters should come next order to write sonnets of its own. In order for the model to be able to generate words, it needs to be able to include white spaces between non-white space characters. 

----

### Use our data tool to create X and Y splits

You'll need the `create_X_and_Y` method for this task. 

In [ ]:
# TODO: provide a walk through of data_cleaning_toolkit with unit tests that check for understanding 
X, y = dctk.create_X_and_Y()

![](https://miro.medium.com/max/891/0*jGB1CGQ9HdeUwlgB)

In [ ]:
# notice that our input matrix isn't actually a matrix - it's a rank 3 tensor
X.shape

In $X$.shape we see three numbers (*n1*, *n2*, *n3*). What do these numbers mean?

Well, *n1* tells us the number of samples that we have. But what about the other two?

In [ ]:
# first index returns a signle sample, which we can see is a sequence 
first_sample_index = 0 
X[first_sample_index]

Notice that each sequence (i.e. $X[i]$ where $i$ is some index value) is `maxlen` long and has `dctk.n_features` number of features. Let's try to better understand this shape. 

In [ ]:
# each sequence is maxlen long and has dctk.n_features number of features
X[first_sample_index].shape

**Each row corresponds to a character vector** and there are `maxlen` number of character vectors. 

**Each column corresponds to a unique character** and there are `dctk.n_features` number of features. 


In [ ]:
# let's index for a single character vector 
first_char_vect_index = 0
X[first_sample_index][first_char_vect_index]

Notice that there is a single `TRUE` value and all the rest of the values are `FALSE`. 

This is a one-hot encoding for which character appears at each index within a sequence. Specifically, the cell above is looking at the first character in the sequence.

Only a single character can appear as the first character in a sequence, so there will necessarily be a single `TRUE` value and the rest will be `FALSE`. 

Let's say that `TRUE` appears in the $ith$ index; by  $ith$ index we simply mean some index in the general case. How can we find out which character that actually corresponds to?

To answer this question, we need to use the character-to-integer look up dictionaries. 

In [ ]:
# take a look at the index to character dictionary
# if a TRUE appears in the 0th index of a character vector,
# then we know that whatever char you see below next to the 0th key 
# is the character that that character vector is endcoding for
dctk.int_char

In [ ]:
# let's look at an example to tie it all together

seq_len_counter = 0

# index for a single sample 
for seq_of_char_vects in X[first_sample_index]:
    
    # get index with max value, which will be the one TRUE value 
    index_with_TRUE_val = np.argmax(seq_of_char_vects)
    
    print (dctk.int_char[index_with_TRUE_val])
    
    seq_len_counter+=1
    
print ("Sequence length: {}".format(seq_len_counter))

## Time for Questions 

----
**Question 1:** 

In your own words, how would you describe the numbers from the shape print out of `X.shape` to a fellow classmate?


**Answer 1:**

`X.shape` is a rank 3 tensor. 

Each of the numbers in this shape can be summerized as follows: 

`X.shape = (num of seqs, len of seqs, num of features)`

`num of seqs`: number of sequence samples that we have created

`len of seqs`: length of the sequences (i.e. number of characters in sequence)

`num of features`: number of features for our model to predict (i.e. number of unique characters)

----


### Build a Text Generation model

Now that we have prepped our data (and understood that process) let's finally build out our character generation model, similar to what we did in the guided project.

In [ ]:
def sample(preds, temperature=1.0):
    """
    Helper function to sample an index from a probability array
    """
    # convert preds to array 
    preds = np.asarray(preds).astype('float64')
    # scale values 
    preds = np.log(preds) / temperature
    # exponentiate values
    exp_preds = np.exp(preds)
    # this equation should look familar to you (hint: it's an activation function)
    preds = exp_preds / np.sum(exp_preds)
    # Draw samples from a multinomial distribution
    probas = np.random.multinomial(1, preds, 1)
    # return the index that corresponds to the max probability 
    return np.argmax(probas)

def on_epoch_end(epoch, _):
    """"
    Function invoked at end of each epoch. Prints the text generated by our model.
    """
    
    print()
    print('----- Generating text after Epoch: %d' % epoch)
    

    # randomly pick a starting index 
    # will be used to take a random sequence of chars from `text`
    start_index = random.randint(0, len(text) - dctk.maxlen - 1)
    
    # this is our seed string (i.e. input seqeunce into the model)
    generated = ''

    # start the sentence at index `start_index` and include the next` dctk.maxlen` number of chars
    sentence = text[start_index: start_index + dctk.maxlen]

    # add to generated
    generated += sentence

    
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)
    
    # use model to predict what the next 40 chars should be that follow the seed string
    for i in range(40):

        # shape of a single sample in a rank 3 tensor 
        x_dims = (1, dctk.maxlen, dctk.n_features)
        # create an array of zeros with shape x_dims
        # recall that python considers zeros and boolean FALSE as the same
        x_pred = np.zeros(x_dims)

        # create a seq vector for our randomly select sequence 
        # i.e. create a numerical encoding for each char in the sequence 
        for t, char in enumerate(sentence):
            # for sample 0 in seq index t and character `char` encode a 1 (which is the same as a TRUE)
            x_pred[0, t, dctk.char_int[char]] = 1

        # next, take the seq vector and pass into model to get a prediction of what the next char should be 
        preds = model.predict(x_pred, verbose=0)[0]
        # use the sample helper function to get index for next char 
        next_index = sample(preds)
        # use look up dict to get next char 
        next_char = dctk.int_char[next_index]

        # append next char to sequence 
        sentence = sentence[1:] + next_char 
        
        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()

In [ ]:
# need this for on_epoch_end()
text = " ".join(clean_sonnets)

In [ ]:
# create callback object that will print out text generation at the end of each epoch 
# use for real-time monitoring of model performance
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

----
### Train Model

Build a text generation model using LSTMs. Feel free to reference the model used in the guided project. 

It is recommended that you train this model to at least 50 epochs (but more if you're computer can handle it). 

You are free to change up the architecture as you wish. 

Just in case you have difficultly training a model, there is a pre-trained model saved to a file called `trained_text_gen_model.h5` that you can load in (the same way that you learned how to load in Keras models in Sprint 2 Module 4). 

In [ ]:
# build text generation model layer by layer 
# fit model

# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# save trained model to file 
model.save("trained_text_gen_model.h5")

### Let's play with our trained model 

Now that we have a trained model that, though far from perfect, is able to generate actual English words, we can take a look at the predictions to continue to learn more about how a text generation model works. 

We can also take this as an opportunity to unpack the `def on_epoch_end` function to better understand how it works. 

In [ ]:
# this is our joinned clean sonnet data
text

In [ ]:
# randomly pick a starting index 
# will be used to take a random sequence of chars from `text`
# run this cell a few times and you'll see `start_index` is random
start_index = random.randint(0, len(text) - dctk.maxlen - 1)
start_index

In [ ]:
# next use the randomly selected starting index to sample a sequence from the `text`

# this is our seed string (i.e. input seqeunce into the model)
generated = ''

# start the sentence at index `start_index` and include the next` dctk.maxlen` number of chars
sentence = text[start_index: start_index + dctk.maxlen]

# add to generated
generated += sentence

generated

In [ ]:
# this block of code let's us know what the seed string is 
# i.e. the input seqeunce into the model
print('----- Generating with seed: "' + sentence + '"')
sys.stdout.write(generated)

In [ ]:
# use model to predict what the next 40 chars should be that follow the seed string
for i in range(40):

    # shape of a single sample in a rank 3 tensor 
    x_dims = (1, dctk.maxlen, dctk.n_features)
    # create an array of zeros with shape x_dims
    # recall that python considers zeros and boolean FALSE as the same
    x_pred = np.zeros(x_dims)

    # create a seq vector for our randomly select sequence 
    # i.e. create a numerical encoding for each char in the sequence 
    for t, char in enumerate(sentence):
        # for sample 0 in seq index t and character `char` encode a 1 (which is the same as a TRUE)
        x_pred[0, t, dctk.char_int[char]] = 1

    # next, take the seq vector and pass into model to get a prediction of what the next char should be 
    preds = model.predict(x_pred, verbose=0)[0]
    # use the sample helper function to get index for next char 
    next_index = sample(preds)
    # use look up dict to get next char 
    next_char = dctk.int_char[next_index]

    # append next char to sequence 
    sentence = sentence[1:] + next_char 

In [ ]:
# this is the seed string
generated

In [ ]:
# these are the 40 chars that the model thinks should come after the seed stirng
sentence

In [ ]:
# how put it all together
generated + sentence

# Resources and Stretch Goals

## Stretch goals:
- Refine the training and generation of text to be able to ask for different genres/styles of Shakespearean text (e.g. plays versus sonnets)
- Train a classification model that takes text and returns which work of Shakespeare it is most likely to be from
- Make it more performant! Many possible routes here - lean on Keras, optimize the code, and/or use more resources (AWS, etc.)
- Revisit the news example from class, and improve it - use categories or tags to refine the model/generation, or train a news classifier
- Run on bigger, better data

## Resources:
- [The Unreasonable Effectiveness of Recurrent Neural Networks](https://karpathy.github.io/2015/05/21/rnn-effectiveness/) - a seminal writeup demonstrating a simple but effective character-level NLP RNN
- [Simple NumPy implementation of RNN](https://github.com/JY-Yoon/RNN-Implementation-using-NumPy/blob/master/RNN%20Implementation%20using%20NumPy.ipynb) - Python 3 version of the code from "Unreasonable Effectiveness"
- [TensorFlow RNN Tutorial](https://github.com/tensorflow/models/tree/master/tutorials/rnn) - code for training a RNN on the Penn Tree Bank language dataset
- [4 part tutorial on RNN](http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/) - relates RNN to the vanishing gradient problem, and provides example implementation
- [RNN training tips and tricks](https://github.com/karpathy/char-rnn#tips-and-tricks) - some rules of thumb for parameterizing and training your RNN